In [1]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import string
from sklearn import metrics

import matplotlib.pyplot as plt

import warnings  
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [2]:
%ls

 El volumen de la unidad C es Local Disk
 El número de serie del volumen es: 0671-FAEA

 Directorio de C:\Users\Monica\Proyecto Final Ironhack

13/03/2024  13:18    <DIR>          .
13/03/2024  13:18    <DIR>          ..
13/03/2024  12:17    <DIR>          .ipynb_checkpoints
12/03/2024  14:21        30.308.369 LaLiga.ipynb
13/03/2024  13:04             1.092 LaLiga_Classification.csv
12/03/2024  14:20             2.350 LaLiga_Classification_final.csv
13/03/2024  12:46             2.343 LaLiga_Classification_final_draws.csv
13/03/2024  12:42             2.358 LaLiga_Classification_final_goals_conceded.csv
13/03/2024  12:44             2.356 LaLiga_Classification_final_goals_scored.csv
13/03/2024  12:52             2.343 LaLiga_Classification_final_losses.csv
13/03/2024  12:55             2.341 LaLiga_Classification_final_wins.csv
13/03/2024  12:47        30.309.564 LaLiga_draws.ipynb
13/03/2024  12:43        30.309.220 LaLiga_goals_conceded.ipynb
13/03/2024  12:44        30.313.935 LaLi

In [3]:
teams_24_completed = pd.read_csv('spain-la-liga-teams-2023-to-2024-stats (2).csv')
teams_24_completed

,team_name,common_name,season,country,matches_played,matches_played_home,matches_played_away,suspended_matches,wins,wins_home,wins_away,draws,draws_home,draws_away,losses,losses_home,losses_away,points_per_game,points_per_game_home,points_per_game_away,league_position,league_position_home,league_position_away,performance_rank,goals_scored,goals_conceded,goal_difference,total_goal_count,total_goal_count_home,total_goal_count_away,goals_scored_home,goals_scored_away,goals_conceded_home,goals_conceded_away,goal_difference_home,goal_difference_away,minutes_per_goal_scored,minutes_per_goal_scored_home,minutes_per_goal_scored_away,minutes_per_goal_conceded,minutes_per_goal_conceded_home,minutes_per_goal_conceded_away,clean_sheets,clean_sheets_home,clean_sheets_away,btts_count,btts_count_home,btts_count_away,fts_count,fts_count_home,fts_count_away,first_team_to_score_count,first_team_to_score_count_home,first_team_to_score_count_away,corners_total,corners_total_home,corners_total_away,cards_total,cards_total_home,cards_total_away,average_possession,average_possession_home,average_possession_away,shots,shots_home,shots_away,shots_on_target,shots_on_target_home,shots_on_target_away,shots_off_target,shots_off_target_home,shots_off_target_away,fouls,fouls_home,fouls_away,goals_scored_half_time,goals_scored_half_time_home,goals_scored_half_time_away,goals_conceded_half_time,goals_conceded_half_time_home,goals_conceded_half_time_away,goal_difference_half_time,goal_difference_half_time_home,goal_difference_half_time_away,leading_at_half_time,leading_at_half_time_home,leading_at_half_time_away,draw_at_half_time,draw_at_half_time_home,draw_at_half_time_away,losing_at_half_time,losing_at_half_time_home,losing_at_half_time_away,points_per_game_half_time,points_per_game_half_time_home,points_per_game_half_time_away,average_total_goals_per_match,average_total_goals_per_match_home,average_total_goals_per_match_away,goals_scored_per_match,goals_scored_per_match_home,goals_scored_per_match_away,goals_conceded_per_match,goals_conceded_per_match_home,goals_conceded_per_match_away,total_goals_per_match_half_time,total_goals_per_match_half_time_home,total_goals_per_match_half_time_away,goals_scored_per_match_half_time,goals_scored_per_match_half_time_home,goals_scored_per_match_half_time_away,goals_conceded_per_match_half_time,goals_conceded_per_match_half_time_home,goals_conceded_per_match_half_time_away,over05_count,over15_count,over25_count,over35_count,over45_count,over55_count,over05_count_home,over15_count_home,over25_count_home,over35_count_home,over45_count_home,over55_count_home,over05_count_away,over15_count_away,over25_count_away,over35_count_away,over45_count_away,over55_count_away,under05_count,under15_count,under25_count,under35_count,under45_count,under55_count,under05_count_home,under15_count_home,under25_count_home,under35_count_home,under45_count_home,under55_count_home,under05_count_away,under15_count_away,under25_count_away,under35_count_away,under45_count_away,under55_count_away,over05_percentage,over15_percentage,over25_percentage,over35_percentage,over45_percentage,over55_percentage,over05_percentage_home,over15_percentage_home,over25_percentage_home,over35_percentage_home,over45_percentage_home,over55_percentage_home,over05_percentage_away,over15_percentage_away,over25_percentage_away,over35_percentage_away,over45_percentage_away,over55_percentage_away,under05_percentage,under15_percentage,under25_percentage,under35_percentage,under45_percentage,under55_percentage,under05_percentage_home,under15_percentage_home,under25_percentage_home,under35_percentage_home,under45_percentage_home,under55_percentage_home,under05_percentage_away,under15_percentage_away,under25_percentage_away,under35_percentage_away,under45_percentage_away,under55_percentage_away,over05_count_half_time,over15_count_half_time,over25_count_half_time,over05_count_half_time_home,over15_count_half_time_home,over25_count_half_time_home,over05_count_half_time_away,over1

In [4]:
# Crear una lista con todos los DataFrames
dataframes = [teams_24_completed]

# Ordenar cada DataFrame en función de 'league_position' en orden ascendente
for df in dataframes:
    df.sort_values(by='league_position', inplace=True)

# Verificar si están ordenados correctamente
for i, df in enumerate(dataframes, start=1):
    print(f"t_{i}_{i+1} sorted by league_position:\n", df[['team_name', 'league_position']])
    print()

t_1_2 sorted by league_position:
                        team_name  league_position
2                 Real Madrid CF                1
19                     Girona FC                2
1                   FC Barcelona                3
4        Club Atlético de Madrid                4
10          Athletic Club Bilbao                5
13       Real Sociedad de Fútbol                6
9            Real Betis Balompié                7
12                   Valencia CF                8
5                  UD Las Palmas                9
11                    CA Osasuna               10
0                  Villarreal CF               11
15         Getafe Club de Fútbol               12
6               Deportivo Alavés               13
3                     Sevilla FC               14
17  Real Club Deportivo Mallorca               15
14                Rayo Vallecano               16
7        Real Club Celta de Vigo               17
18                      Cádiz CF               18
8               

In [5]:
teams_24_completed['points'] = teams_24_completed['wins'] * 3 + teams_24_completed['draws']
teams_24_completed['points_home'] = teams_24_completed['wins_home'] * 3 + teams_24_completed['draws_home']
teams_24_completed['points_away'] = teams_24_completed['wins_away'] * 3 + teams_24_completed['draws_away']

columns = list(teams_24_completed.columns)
columns.remove('points')
columns.remove('points_home')
columns.remove('points_away')

# Insertamos 'points' en la posición deseada
columns.insert(columns.index('wins'), 'points')
columns.insert(columns.index('wins'), 'points_home')
columns.insert(columns.index('wins'), 'points_away')

# Reasignamos las columnas al DataFrame
teams_24_completed = teams_24_completed[columns]

print(teams_24_completed)

                       team_name           common_name     season country  \
2                 Real Madrid CF           Real Madrid  2023/2024   Spain   
19                     Girona FC             Girona FC  2023/2024   Spain   
1                   FC Barcelona          FC Barcelona  2023/2024   Spain   
4        Club Atlético de Madrid       Atlético Madrid  2023/2024   Spain   
10          Athletic Club Bilbao  Athletic Club Bilbao  2023/2024   Spain   
13       Real Sociedad de Fútbol         Real Sociedad  2023/2024   Spain   
9            Real Betis Balompié            Real Betis  2023/2024   Spain   
12                   Valencia CF           Valencia CF  2023/2024   Spain   
5                  UD Las Palmas         UD Las Palmas  2023/2024   Spain   
11                    CA Osasuna            CA Osasuna  2023/2024   Spain   
0                  Villarreal CF            Villarreal  2023/2024   Spain   
15         Getafe Club de Fútbol             Getafe CF  2023/2024   Spain   

In [6]:
import pandas as pd

# Suponiendo que tienes un DataFrame llamado teams_24

# Seleccionar las columnas requeridas
selected_columns = ['team_name', 'matches_played', 'wins', 'draws', 'losses', 
                    'goals_scored', 'goals_conceded', 'goal_difference', 
                    'total_goal_count', 'points', 'points_per_game']

# Crear un nuevo DataFrame con las columnas seleccionadas
LaLiga_Classification = teams_24_completed[selected_columns]

# Guardar el nuevo DataFrame como un archivo CSV
LaLiga_Classification.to_csv('LaLiga_Classification.csv', index=False)


In [7]:
%ls

 El volumen de la unidad C es Local Disk
 El número de serie del volumen es: 0671-FAEA

 Directorio de C:\Users\Monica\Proyecto Final Ironhack

13/03/2024  13:18    <DIR>          .
13/03/2024  13:18    <DIR>          ..
13/03/2024  12:17    <DIR>          .ipynb_checkpoints
12/03/2024  14:21        30.308.369 LaLiga.ipynb
13/03/2024  13:20             1.092 LaLiga_Classification.csv
12/03/2024  14:20             2.350 LaLiga_Classification_final.csv
13/03/2024  12:46             2.343 LaLiga_Classification_final_draws.csv
13/03/2024  12:42             2.358 LaLiga_Classification_final_goals_conceded.csv
13/03/2024  12:44             2.356 LaLiga_Classification_final_goals_scored.csv
13/03/2024  12:52             2.343 LaLiga_Classification_final_losses.csv
13/03/2024  12:55             2.341 LaLiga_Classification_final_wins.csv
13/03/2024  12:47        30.309.564 LaLiga_draws.ipynb
13/03/2024  12:43        30.309.220 LaLiga_goals_conceded.ipynb
13/03/2024  12:44        30.313.935 LaLi

In [17]:
import pandas as pd

# Cargar los archivos CSV en DataFrames
LaLiga_Classification_final = pd.read_csv('LaLiga_Classification_final.csv')
LaLiga_Classification_final_wins = pd.read_csv('LaLiga_Classification_final_wins.csv')
LaLiga_Classification_final_losses = pd.read_csv('LaLiga_Classification_final_losses.csv')
LaLiga_Classification_final_draws = pd.read_csv('LaLiga_Classification_final_draws.csv')
LaLiga_Classification_final_goals_scored = pd.read_csv('LaLiga_Classification_final_goals_scored.csv')
LaLiga_Classification_final_goals_conceded = pd.read_csv('LaLiga_Classification_final_goals_conceded.csv')

In [20]:
import pandas as pd

# Concatenar los DataFrames
LaLiga_Prediction = pd.concat([LaLiga_Classification_final, LaLiga_Classification_final_wins, 
                               LaLiga_Classification_final_losses, LaLiga_Classification_final_draws, 
                               LaLiga_Classification_final_goals_scored, LaLiga_Classification_final_goals_conceded], 
                              axis=1)

# Eliminar columnas duplicadas
LaLiga_Prediction = LaLiga_Prediction.loc[:,~LaLiga_Prediction.columns.duplicated()]
LaLiga_Prediction.index = range(1, len(LaLiga_Prediction) + 1)

# Calcula la diferencia de goles entre expected_goals_scored y expected_goals_conceded
LaLiga_Prediction['expected_difference_goals'] = (LaLiga_Prediction['expected_goals_scored'] 
                                                            - LaLiga_Prediction['expected_goals_conceded'])

# Reorganiza las columnas
LaLiga_Prediction = LaLiga_Prediction[['actual_classification', 'matches_played', 'wins', 'draws', 'losses',
                                                           'goals_scored', 'goals_conceded', 'goal_difference', 'total_goal_count',
                                                           'points', 'points_per_game', 'expected_points', 'expected_wins', 
                                                           'expected_losses', 'expected_draws', 'expected_goals_scored', 
                                                           'expected_goals_conceded', 'expected_difference_goals', 'Image_URL']]

# Mostrar el DataFrame resultante
LaLiga_Prediction

,actual_classification,matches_played,wins,draws,losses,goals_scored,goals_conceded,goal_difference,total_goal_count,points,points_per_game,expected_points,expected_wins,expected_losses,expected_draws,expected_goals_scored,expected_goals_conceded,expected_difference_goals,Image_URL
1,Real Madrid CF,28,21,6,1,60,18,42,78,69,2.46,90,27,1,8,67,24,43,https://cdn.ssref.net/req/202312151/tlogo/fb/5...
2,Girona FC,28,19,5,4,59,33,26,92,62,2.21,82,24,5,7,65,44,21,https://cdn.ssref.net/req/202312151/tlogo/fb/9...
3,FC Barcelona,28,18,7,3,57,34,23,91,61,2.18,81,24,3,9,66,45,21,https://cdn.ssref.net/req/202312151/tlogo/fb/2...
4,Club Atlético de Madrid,28,17,4,7,54,31,23,85,55,1.96,75,22,8,5,60,41,19,https://cdn.ssref.net/req/202312151/tlogo/fb/d...
5,Athletic Club Bilbao,28,15,8,5,48,26,22,74,53,1.89,71,20,7,11,54,35,19,https://cdn.ssref.net/req/202312151/tlogo/fb/2...
6,Real Sociedad de Fútbol,28,11,10,7,40,31,9,71,43,1.54,58,15,9,13,41,41,0,https://cdn.ssref.net/req/202312151/tlogo/fb/e...
7,Real Betis Balompié,28,10,12,6,34,31,3,65,42,1.50,56,14,8,16,39,41,-2,https://cdn.ssref.net/req/202312151/tlogo/fb/f...
8,Valencia CF,27,11,7,9,32,31,1,63,40,1.48,55,16,12,10,37,43,-6,https://cdn.ssref.net/req/202312151/tlogo/fb/d...
9,UD Las Palmas,28,10,7,11,29,31,-2,60,37,1.32,50,14,15,9,33,41,-8,https://cdn.ssref.net/req/202312151/tlogo/fb/0...
10,CA Osasuna,28,10,6,12,31,39,-8,70,36,1.29,48,14,16,8,36,53,-17,https://cdn.ssref.net/req/202312151/tlogo/fb/0...


In [21]:
LaLiga_Prediction.to_csv('LaLiga_Prediction.csv', index=False)